In [1]:
# Tested on Python 3.8.18 
import sys
!{sys.executable} -m pip install taichi==1.7.0  # Install Taichi Lang


In [4]:
import taichi as ti
import math

In [5]:
real = ti.f32
ti.init(default_fp=real, debug=True)

max_steps = 1024
vis_interval = 256
output_vis_interval = 8
steps = 1024

vis_resolution = 1024

n_objects = 3
mass = 1
n_springs = 3
spring_stiffness = 10
damping = 20

scalar = lambda: ti.field(dtype=real)
vec = lambda: ti.Vector.field(2, dtype=real)

loss = scalar()
x = vec()
v = vec()
force = vec()

spring_anchor_a = ti.field(ti.i32)
spring_anchor_b = ti.field(ti.i32)
spring_length = scalar()

dt = 0.001
learning_rate = 5

friction = 0.01


[Taichi] Starting on arch=x64


In [6]:

def allocate_fields():
    ti.root.dense(ti.i, max_steps).dense(ti.j, n_objects).place(x, v, force)
    ti.root.dense(ti.i, n_springs).place(spring_anchor_a, spring_anchor_b,
                                         spring_length)
    ti.root.place(loss)
    ti.root.lazy_grad()

@ti.kernel
def apply_spring_force(t: ti.i32):
    # Kernels can have parameters. there t is a parameter with type int32.
    for i in range(n_springs):  # A parallel for, preferably on GPUs
        a, b = spring_anchor_a[i], spring_anchor_b[i]
        x_a, x_b = x[t - 1, a], x[t - 1, b]
        dist = x_a - x_b
        length = dist.norm() + 1e-4
        F = (length - spring_length[i]) * spring_stiffness * dist / length
        # apply spring impulses to mass points. Use atomic_add for parallel safety.
        ti.atomic_add(force[t, a], -F)
        ti.atomic_add(force[t, b], F)




@ti.kernel
def time_integrate(t: ti.i32):
    for i in range(n_objects):
        s = math.exp(-dt * damping)
        new_v = s * v[t - 1, i] + dt * force[t, i] / mass
        new_x = x[t - 1, i] + dt * new_v
        if new_x[0] > 0.4 and new_v[0] > 0:
            # friction projection
            if new_v[1] > 0:
                new_v[1] -= ti.min(new_v[1], friction * new_v[0])
            if new_v[1] < 0:
                new_v[1] += ti.min(-new_v[1], friction * new_v[0])
            new_v[0] = 0
        v[t, i] = new_v
        x[t, i] = new_x


@ti.kernel
def compute_loss(t: ti.i32):
    x01 = x[t, 0] - x[t, 1]
    x02 = x[t, 0] - x[t, 2]
    area = ti.abs(
        0.5 * (x01[0] * x02[1] - x01[1] * x02[0]))  # area from cross product
    target_area = 0.1
    loss[None] = (area - target_area)**2


def forward():
    for t in range(1, steps):
        apply_spring_force(t)
        time_integrate(t)

    compute_loss(steps - 1)


@ti.kernel
def clear_states():
    for t in range(0, max_steps):
        for i in range(0, n_objects):
            x.grad[t, i] = ti.Vector([0.0, 0.0])
            v.grad[t, i] = ti.Vector([0.0, 0.0])
            force[t, i] = ti.Vector([0.0, 0.0])
            force.grad[t, i] = ti.Vector([0.0, 0.0])


@ti.kernel
def clear_springs():
    for i in range(n_springs):
        spring_length.grad[i] = 0.0


def clear_tensors():
    clear_states()
    clear_springs()


In [8]:
allocate_fields()
x[0, 0] = [0.3, 0.5]
x[0, 1] = [0.3, 0.4]
x[0, 2] = [0.4, 0.4]

spring_anchor_a[0], spring_anchor_b[0], spring_length[0] = 0, 1, 0.1
spring_anchor_a[1], spring_anchor_b[1], spring_length[1] = 1, 2, 0.1
spring_anchor_a[2], spring_anchor_b[2], spring_length[
    2] = 2, 0, 0.1 * 2**0.5

clear_tensors()
forward()

losses = []
for iter in range(25):
    clear_tensors()

    with ti.ad.Tape(loss):
        forward()

    print('Iter=', iter, 'Loss=', loss[None])
    losses.append(loss[None])

    for i in range(n_springs):
        spring_length[i] -= learning_rate * spring_length.grad[i]

for i in range(n_springs):
    print(i, spring_length[i])

clear_tensors()
forward()

[E 11/30/23 12:46:03.429] [snode.cpp:place@44] This variable has been placed.


***********************************
* Taichi Compiler Stack Traceback *
***********************************
/home/taco/miniconda3/envs/taichi/lib/python3.8/site-packages/taichi/core/../lib/taichi_core.so: taichi::Logger::error(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, bool)
/home/taco/miniconda3/envs/taichi/lib/python3.8/site-packages/taichi/core/../lib/taichi_core.so: taichi::lang::SNode::place(taichi::lang::Expr&, std::vector<int, std::allocator<int> > const&)
/home/taco/miniconda3/envs/taichi/lib/python3.8/site-packages/taichi/core/../lib/taichi_core.so(+0x57ecf5) [0x7fb52ce9ccf5]
/home/taco/miniconda3/envs/taichi/lib/python3.8/site-packages/taichi/core/../lib/taichi_core.so(+0x3f4d44) [0x7fb52cd12d44]
/home/taco/miniconda3/envs/taichi/bin/python: PyCFunction_Call
/home/taco/miniconda3/envs/taichi/bin/python: _PyObject_MakeTpCall
/home/taco/miniconda3/envs/ta

RuntimeError: [snode.cpp:place@44] This variable has been placed.